## Rotating information around a ring

The below program sends the rank values around the ring in a loop with #process iterations and sums up all the values that are coming along (sum of all ranks). 

1. Run the program with processes 3,4,5 and check for correct sums.
2. Then substitute MPI_Send with MPI_Ssend (explicit synchronous send). Run the program. You will see a deadlock and you will need to kill the program (with interrupt button ◼).
3. Resolve the deadlock with a serialization. Use the trick: `if rank == `, then do first receive and then send for example. Run the program with 3,4,5 processes, you have resolved the deadlock. Now run the program with 1 process. Does the program finish?
* Why did we use a program with 2 different buffers instead of 1 for the serialization solution?
* Why does the serialized solution still deadlock when running with 1 process?

In [ ]:
?MPI::MPI_Ssend

***
#### C program

In [ ]:
%%file ring.c
#include <stdio.h>
#include <mpi.h>

int main()
{
    int rank, size;
    int snd_buf, rcv_buf;
    int right, left;
    int sum, i;
    MPI_Status status;

    MPI_Init(NULL, NULL);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    right = (rank+1)      % size;
    left  = (rank-1+size) % size;

    sum = 0;
    snd_buf = rank;
    for(i = 0; i < size; i++) 
    {
        MPI_Send(&snd_buf, 1, MPI_INT, right, 17, MPI_COMM_WORLD);
        MPI_Recv(&rcv_buf, 1, MPI_INT, left,  17, MPI_COMM_WORLD, &status);
        snd_buf = rcv_buf;
        sum += rcv_buf;
    }
    printf ("PE%i:\tSum = %i\n", rank, sum);

    MPI_Finalize();
}

In [ ]:
!mpicc ring.c -o ring && mpirun -np 3 --allow-run-as-root ring

***
#### Python program

In [ ]:
%%file ring.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
status = MPI.Status()

right = (rank+1) % size
left = (rank-1+size) % size
sum = 0
snd_buf = rank
rcv_buf = -1

for i in range (0,size):
    comm.send(snd_buf, dest=right)
    rcv_buf = comm.recv(source=left)
    snd_buf = rcv_buf
    sum += rcv_buf

print("PE%i:\tSum = %i" % (rank, sum))

In [ ]:
!mpirun -np 3 --allow-run-as-root python ring.py

***
#### Fortran program

In [ ]:
%%file ring.f90
program ring
use mpi

integer ( kind = 4 ) error
integer :: rank, size
integer :: right, left
integer :: i, sum
integer :: snd_buf, rcv_buf
integer :: status(MPI_STATUS_SIZE)

call MPI_Init(error)
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error)
call MPI_Comm_size(MPI_COMM_WORLD, size, error)
right = mod(rank+1,      size)
left  = mod(rank-1+size, size)
sum = 0
snd_buf = rank
do i = 1, size
    call MPI_Send(snd_buf, 1, MPI_INTEGER, right, 17, MPI_COMM_WORLD, error)
    call MPI_Recv(rcv_buf, 1, MPI_INTEGER, left, 17, MPI_COMM_WORLD, status, error)
    snd_buf = rcv_buf
    sum = sum + rcv_buf
end do
print *, 'PE', rank, ': Sum =', sum
call MPI_Finalize(error)
end

In [ ]:
!mpif90 ring.f90 && mpirun -np 3 --allow-run-as-root a.out

#### You can compare with our solution:

***
#### C serialized solution

In [ ]:
%%file ring.c
#include <stdio.h>
#include <mpi.h>

int main()
{
    int rank, size;
    int snd_buf, rcv_buf;
    int right, left;
    int sum, i;
    MPI_Status status;

    MPI_Init(NULL, NULL);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    right = (rank+1)      % size;
    left  = (rank-1+size) % size;

    sum = 0;
    snd_buf = rank;
    for(i = 0; i < size; i++) 
    {
        if (rank == 0) {
            MPI_Ssend(&snd_buf, 1, MPI_INT, right, 17, MPI_COMM_WORLD);
            MPI_Recv(&rcv_buf, 1, MPI_INT, left,  17, MPI_COMM_WORLD, &status);
        } else {
            MPI_Recv(&rcv_buf, 1, MPI_INT, left,  17, MPI_COMM_WORLD, &status);
            MPI_Ssend(&snd_buf, 1, MPI_INT, right, 17, MPI_COMM_WORLD);
        }
        snd_buf = rcv_buf;
        sum += rcv_buf;
    }
    printf ("PE%i:\tSum = %i\n", rank, sum);

    MPI_Finalize();
}

In [ ]:
!mpicc ring.c -o ring && mpirun -np 3 --allow-run-as-root ring

***
#### Python serialized solution

In [ ]:
%%file ring.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
status = MPI.Status()

right = (rank+1) % size
left = (rank-1+size) % size
sum = 0
snd_buf = rank
rcv_buf = -1

for i in range (0,size):
    if rank == 0:
        comm.ssend(snd_buf, dest=right)
        rcv_buf = comm.recv(source=left)
    else:
        rcv_buf = comm.recv(source=left)
        comm.ssend(snd_buf, dest=right)
    snd_buf = rcv_buf
    sum += rcv_buf

print("PE%i:\tSum = %i" % (rank, sum))

In [ ]:
!mpirun -np 3 --allow-run-as-root python ring.py

***
#### Fortran serialized solution

In [ ]:
%%file ring.f90
program ring
use mpi

integer ( kind = 4 ) error
integer :: rank, size
integer :: right, left
integer :: i, sum
integer :: snd_buf, rcv_buf
integer :: status(MPI_STATUS_SIZE)

call MPI_Init(error)
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error)
call MPI_Comm_size(MPI_COMM_WORLD, size, error)
right = mod(rank+1,      size)
left  = mod(rank-1+size, size)
sum = 0
snd_buf = rank
do i = 1, size
    if (rank == 0) then
        call MPI_Ssend(snd_buf, 1, MPI_INTEGER, right, 17, MPI_COMM_WORLD, error)
        call MPI_Recv(rcv_buf, 1, MPI_INTEGER, left, 17, MPI_COMM_WORLD, status, error)
    else
        call MPI_Recv(rcv_buf, 1, MPI_INTEGER, left, 17, MPI_COMM_WORLD, status, error)
        call MPI_Ssend(snd_buf, 1, MPI_INTEGER, right, 17, MPI_COMM_WORLD, error)
    end if
    snd_buf = rcv_buf
    sum = sum + rcv_buf
end do
print *, 'PE', rank, ': Sum =', sum
call MPI_Finalize(error)
end

In [ ]:
!mpif90 ring.f90 && mpirun -np 3 --allow-run-as-root a.out